<a href="https://colab.research.google.com/github/Mamedova-S/VKR_Mamedova/blob/main/src/audio/preprocess.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# 1 Импорт библиотек

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
!pip freeze > requirements.txt

In [ ]:
!pip install numpy matplotlib scipy sklearn hmmlearn simplejson eyed3 pydub
!pip install pyAudioAnalysis
!pip install plot_metrics
!pip install boto
!pip install wget
!pip install tensorflow keras


# !pip install tensorflow.keras
# !pip install tensorflow.keras.preprocessing.sequence
# !pip install tensorflow.keras.utils
# !pip install tensorflow.keras.preprocessing.text
# !pip install tensorflow.keras.models
# !pip install tensorflow.keras.layers


In [ ]:
import os
import wget
import numpy as np
import pandas as pd
import seaborn as sns
import librosa
import sklearn
import matplotlib.pyplot as plt
import librosa.display


import torch
import torch.nn as nn
import torch.nn.functional as F

from sklearn.metrics import confusion_matrix
%matplotlib inline
import matplotlib.pyplot as plt
from sklearn import preprocessing
from sklearn import metrics

from mpl_toolkits.mplot3d import Axes3D
from sklearn.preprocessing import StandardScaler

from sklearn.preprocessing import LabelEncoder,OneHotEncoder
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.ensemble import GradientBoostingClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestClassifier
from sklearn.tree import DecisionTreeClassifier
from sklearn.naive_bayes import GaussianNB
from sklearn.neighbors import KNeighborsClassifier
from sklearn.svm import SVC
from sklearn.ensemble import AdaBoostClassifier
# from xgboost import XGBClassifier



import fnmatch
import zipfile


import os
import pyAudioAnalysis
from pyAudioAnalysis import audioBasicIO as aIO
from pyAudioAnalysis import audioSegmentation as aS


import scipy.io.wavfile as wavfile
import wave


from numpy.lib import stride_tricks
from PIL import Image
import scipy.io.wavfile as wav

import pandas as pd

import random
import boto


from scipy.io import wavfile as wav
import asyncio

# Metrics
from sklearn import utils
from sklearn.metrics import classification_report, confusion_matrix, roc_curve, roc_auc_score, \
        f1_score, precision_score, recall_score

# Ignore warnings
import warnings
warnings.filterwarnings("ignore")



from __future__ import print_function
import pandas as pd
import boto
import os
import numpy as np
from sklearn.metrics import confusion_matrix
from keras.models import Sequential
from keras.layers import Dense, Dropout, Activation, Flatten
from keras.layers import Conv2D, MaxPooling2D
from keras.utils import np_utils
from keras import backend as K
import matplotlib.pyplot as plt
from sklearn.metrics import roc_curve, auc
import matplotlib.pyplot as plt
from sklearn.metrics import roc_curve, auc
     

# from pyAudioAnalysis.audioBasicIO import read_audio_file


# 2 Загрузка данных

## 2.1 Скачивание набора данных с https://dcapswoz.ict.usc.edu/

In [ ]:
!mkdir DAIC-WOZ
%cd DAIC-WOZ

In [ ]:
!wget -r -np -nH --cut-dirs=3 -R index.html --user=daicwozuser --ask-password --no-check-certificate https://dcapswoz.ict.usc.edu/wwwdaicwoz/

## 2.2 Извлечение данных из архивов

Извлечение WAV-файлов и CSV-файлы расшифровки из ZIP-файлов.

In [ ]:
def extract_files(zip_file, out_dir):

    audio_dir = os.path.join(out_dir, 'audio')
    if not os.path.exists(audio_dir):
        os.makedirs(audio_dir)

    transcripts_dir = os.path.join(out_dir, 'transcripts')
    if not os.path.exists(audio_dir):
        os.makedirs(transcripts_dir)

    zip_ref = zipfile.ZipFile(zip_file)
    for f in zip_ref.namelist(): 
        if f.endswith('.wav'):
            zip_ref.extract(f, audio_dir)
        elif fnmatch.fnmatch(f, '*TRANSCRIPT.csv'):
            zip_ref.extract(f, transcripts_dir)
    zip_ref.close()


In [ ]:
dir_name = '/content/DAIC-WOZ'

out_dir = '/content/drive/MyDrive/Colab Notebooks/VKR_Depression/Dataset/raw'

for file in os.listdir(dir_name):
    if file.endswith('.zip'):
        zip_file = os.path.join(dir_name, file)
        extract_files(zip_file, out_dir)

# 3 Предварительная обработка данных

## 3.1 Удаление тишины

In [ ]:
def remove_silence(filename, out_dir, smoothing=1.0, weight=0.3, plot=False):

    partic_id = 'P' + filename.split('/')[-1].split('_')[0]  # PXXX
    if is_segmentable(partic_id):
        # создать каталог участника для сегментированных файлов wav
        participant_dir = os.path.join(out_dir, partic_id)
        if not os.path.exists(participant_dir):
            os.makedirs(participant_dir)

        os.chdir(participant_dir)

        [Fs, x] = aIO.read_audio_file(filename)
        segments = aS.silence_removal(x, Fs, 0.020, 0.020,
                                     smooth_window=smoothing,
                                     weight=weight,
                                     plot=plot)

        for s in segments:
            seg_name = "{:s}_{:.2f}-{:.2f}.wav".format(partic_id, s[0], s[1])
            wavfile.write(seg_name, Fs, x[int(Fs * s[0]):int(Fs * s[1])])

        # объединить сегментированные волновые файлы в каталоге участников
        concatenate_segments(participant_dir, partic_id)

    

Клипы ниже не были сегментированы из-за чрезмерной статики, близости к виртуальному интервьюеру, уровня громкости и т. д.

In [ ]:
def is_segmentable(partic_id):

    troubled = set(['P300', 'P305', 'P306', 'P308', 'P315', 'P316', 'P343',
                    'P354', 'P362', 'P375', 'P378', 'P381', 'P382', 'P385',
                    'P387', 'P388', 'P390', 'P392', 'P393', 'P395', 'P408',
                    'P413', 'P421', 'P438', 'P473', 'P476', 'P479', 'P490',
                    'P492'])
    return partic_id not in troubled


# 342, 373, 394, 398, 402, 416, 417, 444, 451, 458, 460, 480


In [ ]:
def concatenate_segments(participant_dir, partic_id, remove_segment=True):
    infiles = os.listdir(participant_dir) # список wav-файлов в каталоге
    outfile = '{}_no_silence.wav'.format(partic_id)

    data = []
    for infile in infiles:
        w = wave.open(infile, 'rb')
        data.append([w.getparams(), w.readframes(w.getnframes())])
        w.close()
        if remove_segment:
            os.remove(infile)

    output = wave.open(outfile, 'wb')
    output.setparams(data[0][0])

    for idx in range(len(data)):
        output.writeframes(data[idx][1])
    output.close()

In [ ]:
import pyAudioAnalysis
help(pyAudioAnalysis.audioSegmentation)

In [ ]:

prefix = '/content/drive/MyDrive/Colab Notebooks/VKR_Depression/Dataset/raw/audio/'
if __name__ == '__main__':
    dir_name = prefix
    out_dir = '/content/drive/MyDrive/Colab Notebooks/VKR_Depression/Dataset/raw/segmented_audio/'

    for file in os.listdir(dir_name):
        if file.endswith('.wav'):
            filename = os.path.join(dir_name, file)
            remove_silence(filename, out_dir)
    print("All files saved successfully")
     

All files saved successfully


## 3.2 Создание матриц спектограмм
Создание матрицы спектрограмм из wav-файлов, которые можно передать нейросети CNN


In [ ]:
prefix = '/content/drive/MyDrive/Colab Notebooks/VKR_Depression/Dataset/'

df_train = pd.read_csv(prefix+'train_split_Depression_AVEC2017.csv')
df_test = pd.read_csv(prefix+'dev_split_Depression_AVEC2017.csv')
df_dev = pd.concat([df_train, df_test], axis=0)

In [ ]:
#  Кратковременное преобразование Фурье звукового сигнала.
def stft(sig, frameSize, overlapFac=0.5, window=np.hanning):

    win = window(frameSize)
    hopSize = int(frameSize - np.floor(overlapFac * frameSize))
    samples = np.append(np.zeros((np.floor(frameSize/2.0).astype(int))), sig)
    cols = np.ceil((len(samples) - frameSize) / float(hopSize)) + 1
    cols=cols.astype(int)
    samples = np.append(samples, np.zeros(frameSize))
    frames = stride_tricks.as_strided(samples, shape=(cols, frameSize),
                                      strides=(samples.strides[0]*hopSize,
                                      samples.strides[0])).copy()
    frames *= win

    return np.fft.rfft(frames)


In [ ]:

#  Логарифмически масштабируем ось частоты.


def logscale_spec(spec, sr=44100, factor=20.):

    timebins, freqbins = np.shape(spec)

    scale = np.linspace(0, 1, freqbins) ** factor
    scale *= (freqbins-1)/max(scale)
    scale = np.unique(np.round(scale))
    newspec = np.complex128(np.zeros([timebins, len(scale)]))
    for i in range(0, len(scale)):
        if i == len(scale)-1:
            newspec[:, i] = np.sum(spec[:, int(scale[i]):], axis=1)
        else:
            newspec[:, i] = np.sum(spec[:,int(scale[i]):int(scale[i+1])], axis=1)
    allfreqs = np.abs(np.fft.fftfreq(freqbins*2, 1./sr)[:freqbins+1])
    freqs = []
    for i in range(0, len(scale)):
        if i == len(scale)-1:
            freqs += [np.mean(allfreqs[int(scale[i]):])]
        else:
            freqs += [np.mean(allfreqs[int(scale[i]):int(scale[i+1])])]

    return newspec, freqs

In [ ]:
# Функция, которая преобразует wav-файл в спектрограмму, 
# представленную матрицей, где строки представляют интервалы частот, 
# столбцы представляют время, а значения матрицы представляют интенсивность в децибелах. 
# Матрица этой формы может быть передана в качестве входных данных CNN после нормализации.

def stft_matrix(audiopath, binsize=2**10, png_name='tmp.png',
                save_png=False, offset=0):

    samplerate, samples = wav.read(audiopath)
    s = stft(samples, binsize)
    sshow, freq = logscale_spec(s, factor=1, sr=samplerate)
    ims = 20.*np.log10(np.abs(sshow)/10e-6)  
    timebins, freqbins = np.shape(ims)
    ims = np.transpose(ims)
    ims = np.flipud(ims) 
    if save_png:
        create_png(ims, png_name)

    return ims

In [ ]:
# Сохранить  png спектрограммы.

def create_png(im_matrix, png_name):
    image = Image.fromarray(im_matrix)
    image = image.convert('L') # преобразовать в оттенки серого
    image.save(png_name)


In [ ]:
dir_name = '/content/drive/MyDrive/Colab Notebooks/VKR_Depression/Dataset/raw/segmented_audio/'


# просматривает wav-файлы в каталоге dir_name и создает png-файлы спектрограмм.
# Это визуальное представление того, что передается в CNN до нормализации, 
# хотя на самом деле передается обрезанное матричное представление.
for subdir, dirs, files in os.walk(dir_name):
    for file in files:
        if file.endswith('.wav'):
            wav_file = os.path.join(subdir, file)
            png_name = subdir + '/' + file[:-4] + '.png'
            print('Processing ' + file + '...')
            stft_matrix(wav_file, png_name=png_name, save_png=True)
            

## 3.3 Создание словарей спектограмм
Этот сценарий создает словари для депрессивных и недепрессивных классов с идентификатором каждого участника в качестве ключа и соответствующим сегментированным матричным представлением спектрограммы в качестве значения. Затем указанные значения могут быть обрезаны и выбраны случайным образом в качестве входных данных для CNN.




In [ ]:

# Создает словарь участников с депрессией и участников без депрессии 
# с идентификатором участника в качестве ключа и матричным представлением 
# wav-файла no_silence в качестве значения. Затем эти значения этого словаря 
# случайным образом обрезаются и выбираются для создания 
# сбалансированных входных данных класса и динамика для CNN.

def build_class_dictionaries(dir_name):
    depressed_dict = dict()
    normal_dict = dict()
    for subdir, dirs, files in os.walk(dir_name):
        for file in files:
            if file.endswith('no_silence.wav'):
                partic_id = int(file.split('_')[0][1:])
                if in_dev_split(partic_id):
                    wav_file = os.path.join(subdir, file)
                    # матричное представление спектрограммы
                    mat = stft_matrix(wav_file)
                    depressed = get_depression_label(partic_id)  # 1 if True
                    if depressed:
                        depressed_dict[partic_id] = mat
                    elif not depressed:
                        normal_dict[partic_id] = mat
    return depressed_dict, normal_dict

In [ ]:
# Возвращает True, если участник находится в разделении разработки AVEC 
# (он же участник, для которого у нас есть ярлыки депрессии)

def in_dev_split(partic_id):
    return partic_id in set(df_dev['Participant_ID'].values)


In [ ]:

# Возвращает двоичную метку участника PHQ8. 1 представляет депрессию;
# 0 означает отсутствие депрессии.

def get_depression_label(partic_id):
    return df_dev.loc[df_dev['Participant_ID'] ==
                      partic_id]['PHQ8_Binary'].item()

In [ ]:
if __name__ == '__main__':
    dir_name = '/content/drive/MyDrive/Colab Notebooks/VKR_Depression/Dataset/raw/segmented_audio/'
    depressed_dict, normal_dict = build_class_dictionaries(dir_name)

In [ ]:
print("depressed_dict: ",len(depressed_dict))

depressed_dict:  38


In [ ]:
print("normal_dict:",len(normal_dict))

normal_dict: 84


## 3.4 Random Sampling

Существует большой дисбаланс данных между положительными и отрицательными образцами,что влечет за собой большую погрешность в классификации. Количество недепрессивных субъектов примерно в четыре раза больше, чем у депрессивных. Если использовать эти образцы для обучения, модель будет иметь сильный уклон к недепрессивным данным.
Чтобы решить эту проблему, я выполняю случайную выборку каждой из спектрограмм участников определенной ширины (времени) и постоянной высоты (частоты), чтобы гарантировать, чтобы в CNN было равное соотношение для каждого испытуемого и каждого класса. 


Модуль возвращает 40 случайных 4-секундных спектрограмм для каждого участника. Затем участники из каждого класса случайным образом выбираются в равной пропорции в качестве входных данных для сверточной нейронной сети (CNN). Это был важный шаг в уменьшении систематической ошибки модели.

In [ ]:
np.random.seed(15)  # for reproducibility
# access_key = os.environ['AWS_ACCESS_KEY_ID']
# access_secret_key = os.environ['AWS_SECRET_ACCESS_KEY']


# Находит самый короткий клип во всем наборе данных, который, 
# в соответствии с нашей стратегией случайной выборки, ограничит 
# количество выборок, которые мы берем из каждого клипа, 
# чтобы убедиться, что наши классы сбалансированы.

def determine_num_crops(depressed_dict, normal_dict, crop_width=125):
    merged_dict = dict(normal_dict, **{str(k): v for k, v in depressed_dict.items()})
    shortest_clip = min(merged_dict.items(), key=lambda x: x[1].shape[1])
    shortest_pixel_width = shortest_clip[1].shape[1]
    num_samples_from_clips = shortest_pixel_width / crop_width
    return num_samples_from_clips

In [ ]:
def build_class_sample_dict(segmented_audio_dict, n_samples, crop_width):
    class_samples_dict = dict()
    for partic_id, clip_mat in segmented_audio_dict.items():
            samples = get_random_samples(clip_mat, n_samples, crop_width)
            class_samples_dict[partic_id] = samples
    return class_samples_dict

In [ ]:
def get_random_samples(matrix, n_samples, crop_width):
    clipped_mat = matrix[:, (matrix.shape[1] % crop_width):]
    n_splits = clipped_mat.shape[1] / crop_width
    #print("clipped_mat",type(clipped_mat))
    #print("n_splits",type(n_splits))
    cropped_sample_ls = np.split(clipped_mat, n_splits, axis=1)
    #print("cropped_sample_ls",type(cropped_sample_ls))
    #print("n_samples",type(n_samples))
    samples = random.sample(cropped_sample_ls, int(n_samples))
    return samples


In [ ]:
def create_sample_dicts(crop_width):
    # строим словари участников и сегментированную аудиоматрицу
    # depressed_dict, normal_dict = build_class_dictionaries('/content/drive/MyDrive/Colab Notebooks/VKR_Depression/Dataset/raw/segmented_audio')
    n_samples = determine_num_crops(depressed_dict, normal_dict,
                                    crop_width=crop_width)
    # получить n_sample случайных выборок от каждого депрессивного участника
    depressed_samples = build_class_sample_dict(depressed_dict, n_samples,
                                                crop_width)
    # получить n_sample случайных выборок от каждого участника, не страдающего депрессией
    normal_samples = build_class_sample_dict(normal_dict, n_samples,
                                             crop_width)
    
    # пройдемся по словарям выборок и сохраните файл npz со случайно выбранными n_samples для каждого участника.
    
    # сохранить депрессивные массивы в .npz
    for key, _ in depressed_samples.items():
        path = '/content/drive/MyDrive/Colab Notebooks/VKR_Depression/Dataset/Randomly_Sampled_Data/'
        filename = 'D{}.npz'.format(key)
        outfile = path + filename
        np.savez(outfile, *depressed_samples[key])
    # сохранить недепрессивные массивы .npz
    for key, _ in normal_samples.items():
        path = '/content/drive/MyDrive/Colab Notebooks/VKR_Depression/Dataset/Randomly_Sampled_Data/'
        filename = 'N{}.npz'.format(key)
        outfile = path + filename
        np.savez(outfile, *normal_samples[key])

In [ ]:
# Учитывая обрезанные сегменты от каждого класса и участника, 
# эта функция определяет, сколько образцов мы можем взять от каждого участника 
# и сколько участников мы можем взять от каждого класса.

def rand_samp_train_test_split(npz_file_dir):
    npz_files = os.listdir(npz_file_dir)

    dep_samps = [f for f in npz_files if f.startswith('D')]
    norm_samps = [f for f in npz_files if f.startswith('N')]
    max_samples = min(len(dep_samps), len(norm_samps))
    dep_select_samps = np.random.choice(dep_samps, size=max_samples,
                                        replace=False)
    norm_select_samps = np.random.choice(norm_samps, size=max_samples,
                                         replace=False)
    test_size = 0.2
    num_test_samples = int(len(dep_select_samps) * test_size)

    train_samples = []
    for sample in dep_select_samps[:-num_test_samples]:
        npz_file = npz_file_dir + '/' + sample
        with np.load(npz_file) as data:
            for key in data.keys():
                train_samples.append(data[key])
    for sample in norm_select_samps[:-num_test_samples]:
        npz_file = npz_file_dir + '/' + sample
        with np.load(npz_file) as data:
            for key in data.keys():
                train_samples.append(data[key])
    #y=(np.ones(len(train_samples)//2),np.zeros(len(train_samples)//2))
    #print("y:",y)
    train_labels = np.concatenate((np.ones(len(train_samples)//2),
                                   np.zeros(len(train_samples)//2)))
    test_samples = []
    for sample in dep_select_samps[-num_test_samples:]:
        npz_file = npz_file_dir + '/' + sample
        with np.load(npz_file) as data:
            for key in data.keys():
                test_samples.append(data[key])
    for sample in norm_select_samps[-num_test_samples:]:
        npz_file = npz_file_dir + '/' + sample
        with np.load(npz_file) as data:
            for key in data.keys():
                test_samples.append(data[key])
    test_labels = np.concatenate((np.ones(len(test_samples)//2),
                                  np.zeros(len(test_samples)//2)))

    return np.array(train_samples), train_labels, np.array(test_samples), \
        test_labels

In [ ]:
# построить обрезанные npz-файлы участника, это все no_silence интервью участника, 
# но каждый массив в npz-файлах имеет ширину crop_width

create_sample_dicts(crop_width=125)

# случайная выборка из npz-файлов участников для обеспечения баланса классов
train_samples, train_labels, test_samples, test_labels = rand_samp_train_test_split('/content/drive/MyDrive/Colab Notebooks/VKR_Depression/Dataset/Randomly_Sampled_Data')

# save as npz locally
print("Saving npz file locally...")
np.savez('/content/drive/MyDrive/Colab Notebooks/VKR_Depression/Dataset/Randomly_Sampled_Data/train_samples.npz', train_samples)
np.savez('/content/drive/MyDrive/Colab Notebooks/VKR_Depression/Dataset/Randomly_Sampled_Data/train_labels.npz', train_labels)
np.savez('/content/drive/MyDrive/Colab Notebooks/VKR_Depression/Dataset/Randomly_Sampled_Data/test_samples.npz', test_samples)
np.savez('/content/drive/MyDrive/Colab Notebooks/VKR_Depression/Dataset/Randomly_Sampled_Data/test_labels.npz', test_labels)
print("Saved Locally")

Saving npz file locally...
Saved Locally
